## Data Preprocessing (one2car)
### Import Libraries and Read Data

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

In [2]:
# Read Data
df = pd.read_csv('../WebScrapData/One2Car/Data.csv')
df.head()

,id,name,cost,mile,status,color,gear,web,webid,date
0,1,2015 Mazda 2 1.5 (ปี 09-14) Sports Maxx Sports...,"245,000",140000 km,รถมือสอง,สีขาว,เกียร์อัตโนมัติ,one2car,mazda-2-sports-maxx-sports-กรุงเทพและปริมณฑล-ม...,2023-10-25
1,2,2012 Mazda 3 2.0 (ปี 11-14) Maxx Sports Hatchback,"269,000",82000 km,รถมือสอง,สีน้ำตาล,เกียร์อัตโนมัติ,one2car,mazda-3-maxx-sports-กรุงเทพและปริมณฑล-มีนบุรี/...,2023-10-25
2,3,2021 Mazda 2 1.3 (ปี 15-25) 1.3 S Leather Seda...,"390,000",35 - 40K km,รถมือสอง,สีเทา,เกียร์อัตโนมัติ,one2car,mazda-2-s-leather-ภาคอีสาน-อำเภอเมืองร้อยเอ็ด/...,2023-10-25
3,4,2021 Mazda CX-30 2.0 (ปี 20-25) 2.0 SP SUV AT,"650,000",80 - 85K km,รถมือสอง,สีแดง,เกียร์อัตโนมัติ,one2car,mazda-cx-30-sp-กรุงเทพและปริมณฑล-อำเภอบางพลี/1...,2023-10-25
4,5,2016 Mazda 2 1.3 (ปี 15-25) High Connect Sedan,"357,900",150 - 155K km,รถมือสอง,สีแดง,เกียร์อัตโนมัติ,one2car,mazda-2-high-connect-กรุงเทพและปริมณฑล-กาญจนาภ...,2023-10-25


### Change data type
Change 'cost' columns

In [3]:
df['cost'] = df['cost'].apply(lambda x : x.replace(',',''))
df['cost'] = df['cost'].astype('int64')

Change 'mile' columns  
This column has 2 type
1. X km
2. X - Y km
  
we will change second type to (X+Y)/2 km

In [4]:
def change_mile_dtypes(mile_data):
    try:
        mile_data = mile_data.replace('km','')
        if '-' not in mile_data:    # First type
            return int(mile_data)
        else:                       # Second type
            mile_data = mile_data.replace('K','')
            mile_data = mile_data.split('-')
            return int((int(mile_data[0]) + int(mile_data[1]))/2*1000)
    except:
        return np.NaN
                
df['mile'] = df['mile'].apply(change_mile_dtypes)

# Fill missing with mean
mile_mean = int(df['mile'].mean())
df.fillna(value=mile_mean, inplace=True)
df['mile'] = df['mile'].astype('int64')

Change 'date' columns to datetime type

In [5]:
df['date'] = pd.to_datetime(df['date'])

### Extract 'name' column

Change 'name' columns to
- car_year ex. 2018
- brand ex. Mazda
- model ex. 2, CX-30
- sub_model ex. 1.3 1.5
- sub_model_name ex. High Connect
- model_year_start ex. 9
- model_year_end ex. 14
- car_type ex. Sedan, Hatchback


In [6]:
# Cut from name
def cut_from_name(new_serie, name_serie):
    new_names = []
    for i in range(len(new_serie)):
        name_serie_tokens = name_serie[i].split(' ')
        new_serie_tokens = str(new_serie[i]).split(' ')
        new_name = ''
        for token in name_serie_tokens:
            if token not in new_serie_tokens:
                new_name += token + ' '
        new_names.append(new_name)
    return pd.Series(new_names)

In [7]:
# car_year
df['car_year'] = df['name'].apply(lambda x : int(x[:4]))

# brand
df['brand'] = df['name'].apply(lambda x : 'Mazda' if 'Mazda' in x else 'Other')

# model
models = ['2', '3', '121', '323', 'BT-50', 'BT-50 PRO', 'CX-3', 'CX-5', 'CX-7', 'CX-8', 'CX-9', 'CX-30', 'Familia', 'Fighter', 'Magnum Thunder', 'MX-5', 'RX-7', 'RX-8', 'Savanna']
def check_model(name):
    name = name.split(' ')
    for i in range(len(name)):
        if name[i] == 'Mazda' and ((name[i+1]+' '+name[i+2]) in models):
            return name[i+1]+' '+name[i+2]
        elif name[i] == 'Mazda' and (name[i+1] in models):
            return name[i+1]
    return np.NaN
df['model'] = df['name'].apply(check_model)

# sub_model
sub_models = ['1.1', '1.3', '1.4', '1.5', '1.6', '1.8', '1.9', '2.0', '2.2', '2.3', '2.5', '2.9', '3.2', '3.7']
def check_sub_model(name):
    name = name.split(' ')
    for i in range(len(name)):
        if name[i] in sub_models:
            return name[i]
    return np.NaN
df['sub_model'] = df['name'].apply(check_sub_model)

# car_type
car_types = ['Sedan', 'Pickup', 'Hatchback', 'SUV', 'Convertible']
def check_car_type(name):
    name = name.split(' ')
    for i in range(len(name)):
        if name[i] in car_types:
            return name[i]
    return np.NaN
df['car_type'] = df['name'].apply(check_car_type)

# model_year
def check_model_year(name):
    if 'ปี' in name:
        model_year = name[name.find('(') : name.find(')')+1]
        return model_year
    return np.NaN
df['model_year'] = df['name'].apply(check_model_year)
df['model_year_start'] = df['model_year'].apply(lambda x : int(x[x.find('-')-2 : x.find('-')]) if not pd.isna(x) else np.NaN)
df['model_year_end'] = df['model_year'].apply(lambda x : int(x[x.find('-')+1 : x.find('-')+3]) if not pd.isna(x) else np.NaN)

# Cut transmission cause we already have data
df['name'] = df['name'].apply(lambda x : x.replace('AT',''))
df['name'] = df['name'].apply(lambda x : x.replace('MT',''))

# Cut extract data from name
df['name'] = cut_from_name(df['car_year'], df['name'])
df['name'] = cut_from_name(df['brand'], df['name'])
df['name'] = cut_from_name(df['model'], df['name'])
df['name'] = cut_from_name(df['sub_model'], df['name'])
df['name'] = cut_from_name(df['car_type'], df['name'])
df['name'] = cut_from_name(df['model_year'], df['name'])

# sub_model_name
# The rest of name is sub model name
df['name'] = df['name'].apply(lambda x: x.strip())
df.rename(columns={'name' : 'sub_model_name'}, inplace=True)

### Convert columns to make it easy to understand

In [8]:
# Change transmission col
df.rename(columns={'gear' : 'transmission'}, inplace=True)
df['transmission'] = df['transmission'].apply(lambda x : 'AT' if x == 'เกียร์อัตโนมัติ' else 'MT')

# Change color col
color_map = {
    'สีเขียว':'green',
    'สีเงิน':'silver',
    'สีเทา':'gray',
    'สีเหลือง':'yellow',
    'สีแดง':'red',
    'สีขาว':'white',
    'สีครีม':'cream',
    'สีดำ':'black',
    'สีทอง':'gold',
    'สีน้ำเงิน':'blue',
    'สีน้ำตาล':'brown',
    'สีฟ้า':'sky',
    'สีส้ม':'orange',
    'สีอื่นๆ':'other',
}
df['color'] = df['color'].map(color_map)

# Change status
status_map = {
    'รถมือสอง' : 'second_hand'
}
df['status'] = df['status'].map(status_map)

# Change webid
df['webid'] = df['webid'].apply(lambda x : x.strip())

# Reorder columns
cols = ['car_year', 'brand', 'model', 'sub_model', 'sub_model_name', 'car_type', 'transmission', 'model_year_start', 'model_year_end', 'color', 'mile', 'date', 'webid', 'cost']
df = df[cols]

In [9]:
df.head(5)

,car_year,brand,model,sub_model,sub_model_name,car_type,transmission,model_year_start,model_year_end,color,mile,date,webid,cost
0,2015,Mazda,2,1.5,Sports Maxx Sports,Hatchback,AT,9.0,14.0,white,140000,2023-10-25,mazda-2-sports-maxx-sports-กรุงเทพและปริมณฑล-ม...,245000
1,2012,Mazda,3,2.0,Maxx Sports,Hatchback,AT,11.0,14.0,brown,82000,2023-10-25,mazda-3-maxx-sports-กรุงเทพและปริมณฑล-มีนบุรี/...,269000
2,2021,Mazda,2,1.3,S Leather,Sedan,AT,15.0,25.0,gray,37500,2023-10-25,mazda-2-s-leather-ภาคอีสาน-อำเภอเมืองร้อยเอ็ด/...,390000
3,2021,Mazda,CX-30,2.0,SP,SUV,AT,20.0,25.0,red,82500,2023-10-25,mazda-cx-30-sp-กรุงเทพและปริมณฑล-อำเภอบางพลี/1...,650000
4,2016,Mazda,2,1.3,High Connect,Sedan,AT,15.0,25.0,red,152500,2023-10-25,mazda-2-high-connect-กรุงเทพและปริมณฑล-กาญจนาภ...,357900


### Save data to csv

In [10]:
df.to_csv('data_one2car.csv')